# San Francisco Rental Prices Dashboard

In [28]:
 # imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
# import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [29]:
 # Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

#  Import Data

In [30]:
 # Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(Path("Data/sfo_neighborhoods_census_data.csv"), header=0)
# sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()


# Import the necessary CSVs to Pandas DataFrames
file_path1 = Path("Data/neighborhoods_coordinates.csv")
sfo_loc_data = pd.read_csv(file_path1, header=0)
# sfo_data = pd.read_csv(file_path, index_col="year")
sfo_loc_data.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


## Panel Visualizations

In [56]:
# Define Panel Visualization Functions
# Defining function for the Housing Units Per Year to plot the bar chart
#def housing_units_per_year():
sfo_data1 = sfo_data.reset_index()
num_of_housing_unit = sfo_data1[["housing_units", "year"]]
avg_unit_per_year = num_of_housing_unit.groupby("year").sum().reset_index()
    
bar_plot = px.bar(avg_unit_per_year, x="year", y="housing_units", 
                          title="Hosing Unit in San Fransico from 2010 to 2016")
#    return  bar_plot

# Getting the average cost per year
avg_cost =sfo_data.groupby("year").mean().reset_index()
avg_cost =avg_cost[["year","sale_price_sqr_foot","gross_rent"]]
    
# Defining function for  average gsale price per sqr foot per year  to plot the bar chart    
#def average_gross_rent():
sale_price_sqr_foot = avg_cost[["year","sale_price_sqr_foot"]]
    
bar_plot1 = sale_price_sqr_foot.hvplot(x='year', y='sale_price_sqr_foot',
                           title ='AVERAGE PRICE PER SQRFT PER YEAR')
#    return  bar_plot1

 
# Defining function for the Average Gross Rent in San Francisco Per Year to plot the bar char."""
#def average_sales_price():
gross_rent = avg_cost[["year","gross_rent"]]

bar_plot2 = gross_rent.hvplot(x='year', y='gross_rent',title ='AVERAGE GROSS RENT PER YEAR')
# return  bar_plot2

# defining function for the average price by neighborhood 
#def average_price_by_neighborhood():
avg_by_neigh = sfo_data.groupby(["year","neighborhood"]).mean().reset_index()
bar_plot3 =avg_by_neigh.hvplot(x='year', y='sale_price_sqr_foot',groupby ='neighborhood') 
#    return  bar_plot3

#def average_price_by_neighborhood2():
avg_by_neigh = sfo_data.groupby(["year","neighborhood"]).mean().reset_index()
bar_plot3b =avg_by_neigh.hvplot(x='year', y='gross_rent',groupby ='neighborhood')
#return  bar_plot3b

# defining a function for the 10 most expensive neighborhood
#def top_most_expensive_neighborhoods():
#    """Top 10 Most Expensive Neighborhoods."""
sfo_data2 = sfo_data[['neighborhood','sale_price_sqr_foot','housing_units','gross_rent']]
avg_sfo_by_neigbhoor = sfo_data2.groupby(["neighborhood"]).mean().reset_index()
Ten_most_exp=avg_sfo_by_neigbhoor.nlargest(10,['sale_price_sqr_foot']).reset_index()
    # Ploting the Hvplot for the 10 most expensive neighborhood
    
bar_plot4 =Ten_most_exp.hvplot.bar(x='neighborhood',y='sale_price_sqr_foot' ,rot =90 ,height =400
                      ,title ='Top 10 Most Expensive Neigbhoods in SFO')
#    return  bar_plot4

#   defining a function for the Comparison of Rent and Sales Prices of Most Expensive Neighborhoods.""" 
#def most_expensive_neighborhoods_rent_sales():  
avg_sfo_by_year1 = sfo_data.groupby(["year","neighborhood"]).mean().reset_index()   
bar_plot5 = avg_sfo_by_year1.set_index('year').hvplot(kind ='bar',stacked =False,width =800,height =500,
                groupby ='neighborhood',x='housing_units',rot =90)
#return  bar_plot5

# definiing a function for the parllel coordinates    
#def parallel_coordinates():
#   """Parallel Coordinates Plot."""
df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(Ten_most_exp["neighborhood"])]
df_expensive_neighborhoods_per_year1 = df_expensive_neighborhoods_per_year[["sale_price_sqr_foot",
                                                                            "housing_units",
                                                                          "gross_rent"]]
    
bar_plot6=px.parallel_coordinates(df_expensive_neighborhoods_per_year1, color='sale_price_sqr_foot',
                       title ="Parallel Coordinates Analysis of Most Expensive San Fransico Neighborhood")
#    return  bar_plot6


#def parallel_categories():
#    """Parallel Categories Plot."""
sfo_data2 = sfo_data[['neighborhood','sale_price_sqr_foot','housing_units','gross_rent']]
avg_sfo_by_neigbhoor = sfo_data2.groupby(["neighborhood"]).mean().reset_index()
Ten_most_exp=avg_sfo_by_neigbhoor.nlargest(10,['sale_price_sqr_foot']).reset_index()
bar_plot7=px.parallel_categories(Ten_most_exp, 
                       dimensions=["neighborhood","sale_price_sqr_foot", "housing_units","gross_rent"],
                       title ="Parallel Categories Analysis of Most Expensive San Fransico Neighborhood",
                       color='sale_price_sqr_foot')
#    return  bar_plot7


# defining a function for the map
#def neighborhood_map():
#     Calculate the mean values for each neighborhood
sfo_data1= sfo_data[["neighborhood","sale_price_sqr_foot","housing_units","gross_rent"]]
avg_per_neigh =sfo_data1.groupby(["neighborhood"]).mean()
    # Concatenate data - average values with the neighborhood locations
sfo_data_with_loc = (pd.concat([sfo_loc_data, avg_per_neigh], axis=1).dropna().reset_index())
#        # Rename columns to be 'num_sales' and 'num_foreclosures and plot the graph
sfo_data_with_loc
sfo_data_with_loc.columns = ["index","neighborhood","Latitude","Longitude","sale_price_sqr_foot","housing_units","gross_rent"]
map_1 = px.scatter_mapbox(
            sfo_data_with_loc,
            lat="Latitude",
            lon="Longitude",
            color="gross_rent",
            title="Average Sale Price Per Square Foot and Gross Rent In San Francisco"
                )
#return  map_1

# Defining a function for the sunburst chart    
def sunburst():
    df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(Ten_most_exp["neighborhood"])]
    fig =px.sunburst(df_expensive_neighborhoods_per_year,path=['year','neighborhood'],
                 color ='gross_rent',values ='sale_price_sqr_foot',
                 width =800,height =600,
                  color_continuous_scale='Rdbu',
                  title ="")
    #fig.show()
    return  fig


In [63]:
# Put bar plots in row
row_of_bar = pn.Row(bar_plot,bar_plot)

# Put parallel plots in a single row
row_of_parallel = pn.column(bar_plot, map_1)

AttributeError: module 'panel' has no attribute 'column'

In [62]:
# Create tabs
tabs = pn.Tabs(
   # ("All Plots", plots_as_column),
    ("General Plots", row_of_bar),
    ("Statistical Plots", row_of_parallel)
)
tabs

ValueError: Out of range float values are not JSON compliant

Tabs
    [0] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
    [1] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)